<a href="https://colab.research.google.com/github/ics435/hw5_neural_network/blob/master/tox21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

In [1]:
# Download Tox21 data set.
!curl -LO http://bioinf.jku.at/research/DeepTox/tox21_dense_train.csv.gz
!gunzip tox21_dense_train.csv.gz 
!curl -LO http://bioinf.jku.at/research/DeepTox/tox21_labels_train.csv.gz
!gunzip tox21_labels_train.csv.gz
!curl -LO http://bioinf.jku.at/research/DeepTox/tox21_dense_test.csv.gz
!curl -LO http://bioinf.jku.at/research/DeepTox/tox21_labels_test.csv.gz
!gunzip tox21_dense_test.csv.gz
!gunzip tox21_labels_test.csv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.5M  100 18.5M    0     0  1370k      0  0:00:13  0:00:13 --:--:-- 3350k


In [83]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_tr = pd.read_csv('tox21_dense_train.csv')
x_tr = np.array(x_tr.iloc[:,1:])
scaler.fit(x_tr)
x_tr = scaler.transform(x_tr)

y_tr = pd.read_csv('tox21_labels_train.csv')
y_tr = y_tr.fillna(0)
y_tr = np.array(y_tr.loc[:,'NR.AR']) # Each Column is a type of toxicity.

x_te = pd.read_csv('tox21_dense_test.csv')
x_te = np.array(x_te.iloc[:,1:])
x_te = scaler.transform(x_te) # Must use exact same preprocessing as train set.

y_te = pd.read_csv('tox21_labels_test.csv')
y_te = y_te.fillna(0)
y_te = np.array(y_te.loc[:,'NR.AR']) 

print(x_tr.shape, y_tr.shape, x_te.shape, y_te.shape)


(12060, 801) (12060,) (647, 801) (647,)


In [86]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

model = DecisionTreeClassifier(random_state=0, max_depth=10)
#model = MLPClassifier(hidden_layer_sizes=(100, 100), random_state=1, max_iter=100)

# Cross validation.
cv_auc = cross_val_score(model, x_tr, y_tr, cv=3, scoring='roc_auc')
print('ROC AUC on cross-validation:', cv_auc)

# Evaluate on test set.
model.fit(x_tr, y_tr)
p_te = model.predict_proba(x_te)
auc_te = roc_auc_score(y_te, p_te[:, 1])
print("Test set AUC: %3.5f" % (auc_te))


ROC AUC on cross-validation: [0.80800488 0.78119279 0.78204189]
Test set AUC: 0.73491
